In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
zori_df = pd.read_csv('./ZORI_CLEAN.csv')
zillow_counties = pd.read_csv('./Zillow_Data_direct/zillow_homes_zip_county.csv')
zillow_counties.head()
zori_df.head()

,RegionName,Yr_Mo,RI,Year,Month
0,1604,2014-01,1030.0,2014,1
1,1604,2014-02,1034.0,2014,2
2,1604,2014-03,1039.0,2014,3
3,1604,2014-04,1043.0,2014,4
4,1604,2014-05,1048.0,2014,5


In [5]:
def fix_zip(series):
      return series.astype(str).str.replace('[^\w\s]', '').str.zfill(5)

In [8]:
zori_df[['RegionName','Yr_Mo','RI']]
zori_df['RegionName']=fix_zip(zori_df['RegionName'])
zori_df['RegionName']
pvt_tbl = zori_df[['RegionName','Yr_Mo','RI']].pivot_table(columns='RegionName',values='RI',index='Yr_Mo',)
pvt_tbl.reset_index(inplace=True)
pvt_tbl.head()

RegionName,Yr_Mo,01604,01702,01752,01851,01852,02109,02111,02113,02114,...,98402,98403,98405,98406,98422,98466,98498,98661,98683,99207
0,2014-01,1030.0,1644.0,1335.0,988.666667,1232.0,2780.75,3560.0,2201.0,2515.0,...,997.0,868.0,972.0,1014.5,1219.5,1059.0,934.0,910.0,974.0,956.0
1,2014-02,1034.0,1639.0,1339.0,976.000000,1233.5,2778.00,3561.0,2208.0,2512.0,...,1002.0,872.0,979.0,1016.0,1218.0,1061.0,938.0,918.0,982.5,958.0
2,2014-03,1039.0,1634.0,1343.0,986.000000,1235.0,2785.00,3561.5,2215.0,2510.0,...,1007.0,875.0,985.0,1013.0,1222.0,1063.0,942.0,926.0,991.0,961.0
3,2014-04,1043.0,1629.0,1347.0,996.000000,1236.0,2792.00,3562.0,2222.0,2508.0,...,1012.0,878.0,991.0,1010.5,1225.0,1065.0,946.0,934.0,1000.0,964.0
4,2014-05,1048.0,1625.0,1351.0,1005.000000,1238.0,2799.00,3563.0,2228.0,2506.0,...,1017.0,881.0,997.0,1008.0,1228.5,1067.0,950.0,943.0,1008.0,966.0


In [13]:
train_lst = list(pvt_tbl['Yr_Mo'][:-18])
#train_lst

In [14]:
train_df = pvt_tbl[pvt_tbl['Yr_Mo'].isin(train_lst)]
train_df.set_index('Yr_Mo',drop=True,inplace=True)
test_df = pvt_tbl[~pvt_tbl['Yr_Mo'].isin(train_lst)]
test_df.set_index('Yr_Mo',drop=True,inplace=True)

train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')

In [15]:
zillow_counties.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County


In [59]:
county_zips_s = zillow_counties.groupby(['CountyName', 'State'])['RegionName'].unique()
county_zips_df = county_zips_s.to_frame()
county_zips_df.reset_index(inplace=True)

In [60]:
county_zips_df.head()

,CountyName,State,RegionName
0,Abbeville County,SC,"[29620, 29638, 29628, 29639]"
1,Acadia Parish,LA,"[70526, 70578, 70525, 70543, 70559, 70516, 705..."
2,Accomack County,VA,"[23336, 23417, 23421, 23410, 23356, 23420, 234..."
3,Ada County,ID,"[83646, 83709, 83642, 83704, 83706, 83705, 836..."
4,Adair County,IA,"[50849, 50002, 50846, 50858, 50837]"


In [62]:
county_zips_df['RegionCount'] = county_zips_df['RegionName'].transform(lambda x: len(x))
county_zips_df.head()

,CountyName,State,RegionName,RegionCount
0,Abbeville County,SC,"[29620, 29638, 29628, 29639]",4
1,Acadia Parish,LA,"[70526, 70578, 70525, 70543, 70559, 70516, 705...",10
2,Accomack County,VA,"[23336, 23417, 23421, 23410, 23356, 23420, 234...",28
3,Ada County,ID,"[83646, 83709, 83642, 83704, 83706, 83705, 836...",15
4,Adair County,IA,"[50849, 50002, 50846, 50858, 50837]",5


In [63]:
county_zips_df.to_csv('county_zip_count.csv')

In [69]:
states = county_zips_df['State'].unique()

In [71]:
zori_df.head()

,RegionName,Yr_Mo,RI,Year,Month
0,01604,2014-01,1030.0,2014,1
1,01604,2014-02,1034.0,2014,2
2,01604,2014-03,1039.0,2014,3
3,01604,2014-04,1043.0,2014,4
4,01604,2014-05,1048.0,2014,5


In [82]:
zori_zips = zori_df['RegionName'].unique()

In [83]:
zori_zips

array(['01604', '01702', '01752', ..., '98661', '98683', '99207'],
      dtype=object)